##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [2]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.5539868 , -0.02661923, -0.00193739, -0.18131602,  0.35901904,
         0.30089116, -0.9983874 , -0.7464336 ,  0.5097132 , -0.5380107 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.16699743, 0.09344495, 0.09578004, 0.08005194, 0.1374157 ,
        0.12965575, 0.03536082, 0.04549298, 0.15976514, 0.05603525]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.0428724

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [10]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5


   32/60000 [..............................] - ETA: 6:54 - loss: 2.4441 - accuracy: 0.1250

  896/60000 [..............................] - ETA: 17s - loss: 1.7347 - accuracy: 0.4487 

 1792/60000 [..............................] - ETA: 10s - loss: 1.3119 - accuracy: 0.6060

 2720/60000 [>.............................] - ETA: 7s - loss: 1.0945 - accuracy: 0.6754 

...

56896/60000 [===========================>..] - ETA: 0s - loss: 0.0723 - accuracy: 0.9778

57792/60000 [===========================>..] - ETA: 0s - loss: 0.0723 - accuracy: 0.9778

58688/60000 [============================>.] - ETA: 0s - loss: 0.0723 - accuracy: 0.9778

59584/60000 [============================>.] - ETA: 0s - loss: 0.0721 - accuracy: 0.9780

60000/60000 [==============================] - 3s 57us/sample - loss: 0.0722 - accuracy: 0.9780


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 1s - loss: 0.0717 - accuracy: 0.9782


[0.07165977018726989, 0.9782]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.39129213e-07, 2.48362397e-07, 6.81459233e-06, 1.45580198e-04,
        4.99621124e-13, 3.08874678e-06, 2.67206765e-12, 9.99826491e-01,
        1.46831653e-05, 2.92609911e-06],
       [2.01138199e-08, 5.40703814e-03, 9.94537413e-01, 2.29258985e-05,
        1.73300016e-16, 1.13538124e-06, 1.24215404e-07, 2.35344513e-13,
        3.12812590e-05, 4.64853799e-15],
       [8.53231597e-07, 9.98504639e-01, 3.22262174e-04, 9.67663163e-06,
        2.37207914e-05, 1.06307625e-05, 7.22374580e-06, 9.53194976e-04,
        1.67667386e-04, 1.62497088e-07],
       [9.99603808e-01, 9.84166633e-08, 1.00599202e-04, 7.34317837e-06,
        8.62698380e-06, 6.78679498e-05, 7.64336164e-06, 1.62142780e-04,
        3.74241085e-06, 3.81181308e-05],
       [1.40516158e-06, 4.61783003e-08, 2.57583480e-07, 7.00822467e-09,
        9.94257152e-01, 9.91129923e-07, 2.55258055e-06, 5.00702481e-05,
        1.90610444e-05, 5.66859683e-03]], dtype=float32)>